In [1]:
import os
%pwd

'/home/towet/Desktop/OpenProjects/Malaria-Cell-Histopathology-Classification-Transfer-Learning/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/OpenProjects/Malaria-Cell-Histopathology-Classification-Transfer-Learning'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    dataset: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from MalariaCellHistopathologyClassification.constants import *
from MalariaCellHistopathologyClassification.utils.common import read_yaml, create_directories, save_json

2024-03-05 14:45:37.138052: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 14:45:37.198256: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 14:45:37.198354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 14:45:37.199957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 14:45:37.208357: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 14:45:44.488427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/Towet-Tum/Malaria-Cell-Histopathology-Classification-Transfer-Learning.mlflow \
MLFLOW_TRACKING_USERNAME=Towet-Tum \
MLFLOW_TRACKING_PASSWORD=39c66009c3d7d6d659cbc691e65a93c46873d3c4 \
python script.py

In [9]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        
        dataset = os.path.join("artifacts", "data_ingestion", "dataset/")
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/Malaria_cell_TF.h5",
            mlflow_uri="https://dagshub.com/Towet-Tum/Malaria-Cell-Histopathology-Classification-Transfer-Learning.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMG_SHAPE,
            params_batch_size=self.params.BATCH_SIZE,
            dataset=str(dataset)
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from MalariaCellHistopathologyClassification.utils.common import load__process_dataset

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        _,_,test_ds, class_names = load__process_dataset(self.config.dataset)
        self.score = self.model.evaluate(test_ds)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG19-Malaria-Cell")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e